In [2]:
import feedparser
from urllib2 import urlopen
import lxml.html as html
import nltk
import string

In [45]:
d = feedparser.parse('https://lenta.ru/rss/articles')
dataDict = dict()
for e in d.entries:
    request = urlopen(e.link).read().decode('utf-8')
    page = html.document_fromstring(request)
    text = nltk.word_tokenize(''.join(page.xpath("//div[@class='b-text clearfix']/p//text()")))
    text = [word.encode('utf-8').lower() for word in text if word not in string.punctuation
                    and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
    if e.category in dataDict:
        dataDict[e.category] += text
    else:
        dataDict[e.category] = text

In [33]:
def load_ngrams(in_file):
    input_f = open(in_file, "r")
    n_dict = {}
    for line in input_f:
        line = line.replace('\n','').split('\t')
        line = [word for word in line if word not in [''] and word not in string.punctuation]
        n_dict[tuple(line[1:])] = int(line[0])
    input_f.close()
    return n_dict

In [34]:
n2_dict = load_ngrams("2grams-3.txt")

In [35]:
n3_dict = load_ngrams("3grams-3.txt")

In [52]:
import math
def compute_perplexity(category, n3_dict, n2_dict):
    sum_probability = 0
    unknown_words = 0
    min_probability = 1
    sum_log = 0

    for i in range(2, len(category)):
        try:
            current_probability = (n3_dict[(category[i-2], category[i-1], category[i])] + 0.0)/n2_dict[(category[i-2],category[i-1])]
            sum_probability += current_probability
            if current_probability < min_probability:
                min_probability = current_probability
        except KeyError:
            unknown_words +=1
    print (unknown_words, len(category), sum_probability)
    known_words = len(category) - unknown_words
    min_probability /= 10
    probability_to_substract = (unknown_words * min_probability + 0.0) / known_words
    for i in range(2, len(category)):
        try:
            current_probability = (n3_dict[(category[i-2], category[i-1], category[i])] + 0.0)/(n2_dict[(category[i-2], category[i-1])])
            sum_log += math.log(current_probability - probability_to_substract, 2)
        except KeyError:
            sum_log += math.log(min_probability, 2)

    return sum_log

In [60]:
for key,value in dataDict.items():
    sum_log_2 = compute_perplexity(value, n3_dict, n2_dict)
    print("Perplexity for {} = {}".format(key.encode('utf-8'), 2**(-sum_log_2 / len(value))))

(3940, 4624, 12781.013231984643)
Perplexity for 69-я параллель = 26243.4414926
(13028, 15474, 61845.62473183796)
Perplexity for Наука и техника = 50323.0255395
(14480, 17502, 81203.32692778941)
Perplexity for Мир = 36493.4434325
(4801, 5521, 29043.137931312463)
Perplexity for Ценности = 44066.2385491
(10678, 12834, 46052.21317224106)
Perplexity for Финансы = 23285.1819143
(3584, 4525, 18198.70219022509)
Perplexity for Из жизни = 27972.5519146
(9805, 11669, 38699.30409394305)
Perplexity for Путешествия = 50765.65036
(9613, 11906, 57770.93361333205)
Perplexity for Библиотека = 29040.5312117
(15041, 17598, 34282.60132377665)
Perplexity for Культура = 45121.9571557
(32164, 38899, 200529.8689486297)
Perplexity for Россия = 42726.9866872
(16419, 19213, 102035.4801184347)
Perplexity for Силовые структуры = 47469.8308416
(7847, 9295, 48977.88934164327)
Perplexity for Интернет и СМИ = 28872.0009843
(12330, 14639, 60325.93189266505)
Perplexity for Бывший СССР = 51917.5238794
(24185, 28730, 15683